In [ ]:
%matplotlib notebook
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.utils import shuffle
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Conv1D, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from datetime import date, timedelta
import os

import neuralNets

def oneHotEncodeData3Classes(targets):
    j=0
    Y_val = np.zeros((targets.shape[0], 3))
    for j in range(targets.shape[0]):
        if targets[j] == 0:
            Y_val[j, 0] = 1
        elif targets[j] == 1:
            Y_val[j, 1] = 1
        elif targets[j] == 2:
            Y_val[j, 2] = 1
        else:
            print("something went wrong, new class", targets[j])
    return Y_val

labelArray = [0, 0.25, 0.5, 0.75, 1]

In [ ]:
training_data = pd.read_csv("data/numerai_datasets_25.04.21/numerai_training_data.csv")

In [ ]:
feature_cols_wisdom = training_data.columns[training_data.columns.str.startswith('feature_w')]      #264 - 309
feature_cols_consti = training_data.columns[training_data.columns.str.startswith('feature_con')]    #150 - 263
feature_cols_dexter = training_data.columns[training_data.columns.str.startswith('feature_dex')]    #136 - 149
feature_cols_streng = training_data.columns[training_data.columns.str.startswith('feature_str')]    #98  - 135
feature_cols_charis = training_data.columns[training_data.columns.str.startswith('feature_cha')]    #13  - 97
feature_cols_intell = training_data.columns[training_data.columns.str.startswith('feature_int')]    #0   - 12
features = ['feature_w', 'feature_con','feature_dex','feature_str','feature_cha','feature_int']

In [ ]:
era1 = training_data.loc[training_data.era == 'era1']

for feature in features:
    dataArray = []
    for classLabel in labelArray:
        eraClass   = era1.loc[era1.target == classLabel]
    
        feature_cols = eraClass.columns[training_data.columns.str.startswith(feature)]
        feature_data = eraClass[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()#[0:1,:]
        dataArray.append(plotSurfaceOfLabel(feature_data, False))
    n_features = dataArray[0].shape[0]
    feature_count = np.linspace(0, n_features-1, n_features)
    fig = plt.figure()
    for i in range(5):
        plt.plot(feature_count, np.argmax(dataArray[i], axis = 1))

    plt.legend(labelArray)
    plt.show()
    

In [ ]:
n_features = dataArray[0].shape[0]
feature_count = np.linspace(0, n_features-1, n_features)
fig = plt.figure()
for i in range(5):
    plt.plot(feature_count, np.argmax(dataArray[i], axis = 1))

plt.legend(labelArray)
plt.show()

In [ ]:
#plot variance for every class
n_features = dataArray[0].shape[0]
feature_count = np.linspace(0, n_features-1, n_features)

for i in range(5):
    fig = plt.figure()
    plt.plot(feature_count, np.argmax(dataArray[i], axis = 1))

plt.legend(labelArray)
plt.show()

In [ ]:
def plotSurfaceOfLabel(feature_data, render):
    #iterate over features
    n_data, n_features = feature_data.shape
    n_values   = 5
    features = np.zeros((n_features, n_values))
    for j in range(n_data):
        for i in range(n_features):
                curLabel = feature_data[j,i]
                if curLabel in labelArray:
                    features[i,int(curLabel*4)] += 1#/n_data
                else:
                    print(curLabel)
         
    if render:
        featureVector = np.linspace(0,n_features-1, n_features)
        values = np.array(labelArray)

        X, Y = np.meshgrid(values, featureVector)

        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.plot_surface(X, Y, features)
        plt.show()
    
    return features